# Config File Migration
We are proposing going from a process in which config files are maintained
by the OFP team to a process in which the config files are maintained by
the end users. This will not resolve all responsibility of the OFP team
but instead shift there frole from 